# Predicting Medicare Value-Based Payment Program Participation

This project uses a deep learning model to predict whether a hospital participated in the FY 2023 Hospital Value-Based Purchasing (HVBP) Program with the Centers for Medicare & Medicaid Services (CMS). More information about the program can be found at https://www.cms.gov/medicare/quality/initiatives/hospital-quality-initiative/hospital-value-based-purchasing. The training set is limited to hospitals in California due to the greater availability of data from the state compared to the federal government.

In [ ]:
import requests
import warnings
import pandas as pd
import numpy as np
import math
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Load Data

This project requires several datasets.

### CMS Hospital General Information

Use the CMS API to load data from the Hospital General Information dataset, available at https://data.cms.gov/provider-data/dataset/xubh-q36u. The API allows only 500 records to be queried at once. We will review the total number of records, and then paginate through in 500-record chunks.

In [2]:
url = "https://data.cms.gov/provider-data/api/1/datastore/sql?query=%5BSELECT%20COUNT%28%2A%29%20FROM%20dca90d1c-047a-5377-ad18-bb78e1b24050%5D"
response = requests.get(url)
count = int(pd.DataFrame(response.json()).iloc[0,0])
print(f'Number of records: {count:,}')

Number of records: 5,439


In [3]:
number_of_chunks = math.ceil(count / 500)
number_of_chunks

11

In [4]:
cms_hospitals = pd.DataFrame()

for i in range(0,number_of_chunks):
    offset = i * 500
    url = f'https://data.cms.gov/provider-data/api/1/datastore/sql?query=%5BSELECT%20%2A%20FROM%20dca90d1c-047a-5377-ad18-bb78e1b24050%5D%5BLIMIT%20500%20OFFSET%20{offset}%5D;&show_db_columns'
    response = requests.get(url)
    if response.status_code == 200:
        temp = pd.DataFrame(response.json())
        cms_hospitals = pd.concat([cms_hospitals, temp], axis=0, ignore_index=True)
    else:
        print(f'Error: {response.status_code} - {response.text}')

print(f'Number of records: {cms_hospitals.shape[0]:,}')

Number of records: 5,439


In [5]:
cms_hospitals.head(2)

,record_number,facility_id,facility_name,address,citytown,state,zip_code,countyparish,telephone_number,hospital_type,hospital_ownership,emergency_services,meets_criteria_for_promoting_interoperability_of_ehrs,meets_criteria_for_birthing_friendly_designation,hospital_overall_rating,hospital_overall_rating_footnote,mort_group_measure_count,count_of_facility_mort_measures,count_of_mort_measures_better,count_of_mort_measures_no_different,count_of_mort_measures_worse,mort_group_footnote,safety_group_measure_count,count_of_facility_safety_measures,count_of_safety_measures_better,count_of_safety_measures_no_different,count_of_safety_measures_worse,safety_group_footnote,readm_group_measure_count,count_of_facility_readm_measures,count_of_readm_measures_better,count_of_readm_measures_no_different,count_of_readm_measures_worse,readm_group_footnote,pt_exp_group_measure_count,count_of_facility_pt_exp_measures,pt_exp_group_footnote,te_group_measure_count,count_of_facility_te_measures,te_group_footnote
0,1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,Y,3,,7,7,1,5,1,,8,8,2,6,0,,11,11,0,10,1,,8,8,,12,10,
1,6,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,,2,,7,6,0,4,2,,8,7,0,6,1,,11,10,0,10,0,,8,8,,12,11,


### CMS Hospital Value-Based Purchasing (HVBP)

Repeat the approach above to load data from the Hospital Value-Based Purchasing (HVBP) program, available at https://data.cms.gov/provider-data/dataset/ypbt-wvdk.

In [6]:
url = "https://data.cms.gov/provider-data/api/1/datastore/sql?query=%5BSELECT%20COUNT%28%2A%29%20FROM%20ffba8cce-1122-59fa-93d4-2e9a475bb124%5D"
response = requests.get(url)
count = int(pd.DataFrame(response.json()).iloc[0,0])
print(f'Number of records: {count:,}')

Number of records: 2,517


In [7]:
number_of_chunks = math.ceil(count / 500)
number_of_chunks

6

In [8]:
cms_hvbp = pd.DataFrame()

for i in range(0,number_of_chunks):
    offset = i * 500
    url = f'https://data.cms.gov/provider-data/api/1/datastore/sql?query=%5BSELECT%20%2A%20FROM%20ffba8cce-1122-59fa-93d4-2e9a475bb124%5D%5BLIMIT%20500%20OFFSET%20{offset}%5D;&show_db_columns'
    response = requests.get(url)
    if response.status_code == 200:
        temp = pd.DataFrame(response.json())
        cms_hvbp = pd.concat([cms_hvbp, temp], axis=0, ignore_index=True)
    else:
        print(f'Error: {response.status_code} - {response.text}')

print(f'Number of records: {cms_hvbp.shape[0]:,}')

Number of records: 2,517


In [9]:
cms_hvbp.head(2)

,record_number,fiscal_year,facility_id,facility_name,address,citytown,state,zip_code,countyparish,unweighted_normalized_clinical_outcomes_domain_score,weighted_normalized_clinical_outcomes_domain_score,unweighted_person_and_community_engagement_domain_score,weighted_person_and_community_engagement_domain_score,unweighted_normalized_safety_domain_score,weighted_safety_domain_score,unweighted_normalized_efficiency_and_cost_reduction_domain_score,weighted_efficiency_and_cost_reduction_domain_score,total_performance_score
0,1,2023,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,52.000000000000,13.000000000000,Not Available,Not Available,Not Available,Not Available,0.000000000000,0.000000000000,Not Available
1,6,2023,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,40.000000000000,10.000000000000,Not Available,Not Available,Not Available,Not Available,30.000000000000,7.500000000000,Not Available


### California HCAI Licensed Healthcare Facility Listing, December 31, 2022

Download basic details about California health care facilities, including hospitals, from https://data.chhs.ca.gov/dataset/licensed-healthcare-facility-listing.

In [10]:
hcai_facilities = pd.read_csv('https://data.chhs.ca.gov/dataset/59d9abe7-2664-407a-a5aa-f89a866f3381/resource/e93f1582-17be-4289-8efe-2f653246b46f/download/licensed-healthcare-facility-listing-december-31-2022.csv')
print(f'Number of records: {hcai_facilities.shape[0]:,}')

Number of records: 9,594


In [11]:
hcai_facilities.head(2)

,OSHPD_ID,FACILITY_NAME,LICENSE_NUM,FACILITY_LEVEL_DESC,DBA_ADDRESS1,DBA_CITY,DBA_ZIP_CODE,COUNTY_CODE,COUNTY_NAME,ER_SERVICE_LEVEL_DESC,TOTAL_NUMBER_BEDS,FACILITY_STATUS_DESC,FACILITY_STATUS_DATE,LICENSE_TYPE_DESC,LICENSE_CATEGORY_DESC,LATITUDE,LONGITUDE
0,106010735,ALAMEDA HOSPITAL,140000002,Parent Facility,2070 Clinton Ave,Alameda,94501.0,1,Alameda,Emergency - Basic,101,Open,1946-01-01,Hospital,General Acute Care Hospital,37.76266,-122.253991
1,106010739,ALTA BATES SUMMIT MEDICAL CENTER-ALTA BATES CAMPUS,140000004,Parent Facility,2450 Ashby Ave,Berkeley,94705.0,1,Alameda,Emergency - Basic,339,Open,1946-01-02,Hospital,General Acute Care Hospital,37.85645,-122.257430


### California HCAI 2022 Hospital Inpatient - Characteristics by Facility

Download additional details about hospital inpatients from https://data.chhs.ca.gov/dataset/hospital-inpatient-characteristics-by-facility-pivot-profile.

In [12]:
hcai_inpatients = pd.read_excel('https://data.chhs.ca.gov/dataset/ed5d21ce-9ec4-44ca-a482-2f4c767e0528/resource/7bacc638-e70a-4d02-ba25-18ea0874cea2/download/2022pddpivot.xlsx', sheet_name='Data')
print(f'Number of records: {hcai_inpatients.shape[0]:,}')

Number of records: 449


/Users/ellie/opt/anaconda3/envs/tensorflow_env/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [13]:
hcai_inpatients.head(2)

,oshpd_id9,OSHPD_ID,COUNTY,FACILITY_NAME,ADDRESS,CITY,DBA_ZIP_CODE,CONTROL_TYPE_DESC,teach_rural,licensed_bed_size,senate_district_desc,assembly_district_desc,mssa_name,mssa_designation,TOC_Acute,TOC_SN_IC,TOC_Psych,TOC_Chem_Dep,TOC_Rehab,TOC_Unknown,Sex_Male,Sex_Female,Sex_Other_Unknown,Age_0_09,Age_10_19,Age_20_29,Age_30_39,Age_40_49,Age_50_59,Age_60_69,Age_70_79,Age_80_,Age_Other_Unknown,eth_Hispanic,eth_NonHispanic,eth_Other_Unknown,racegrp_aman,racegrp_asian,racegrp_nhpi,racegrp_black,racegrp_white,racegrp_unknown,racegrp_other,racegrp_multirace,Admt_Non_HCF,Admt_Clinic_Dr,Admt_Hosp_Dif,Admt_SNF_ICF_ALF,Admt_Another_HCF,Admt_DistinctPartXfer,Admt_Amb_Surgery,Admt_HospiceFacility,Admt_Court_LE,Admt_NA,Admt_Newborn,admt_disaster_care_site,Route_Your_ER,Route_Another_ER,Route_No_ER,Route_NA,dsch_days,Adm_Emergency,Adm_Urgent,Adm_Elective,Adm_Newborn,Adm_Trauma,Adm_NA,disp_Routine,disp_Acute_Care,disp_Rehab,disp_SN_IC_Care,disp_Residential_Care,disp_Prison_Jail,disp_Against_Medical_Advice,disp_Died,disp_Home_Health_Service,disp_Disaster_Care_Site,disp_Childrens_or_Cancer,disp_Hospice_Care,disp_Psychiatric_Care,disp_CAH,disp_Not_Defined_Elsewhere,disp_Other,disp_Invalid_Blank,DNR_Yes,DNR_No,DNR_Unknown,Payer_Medicare,Payer_Medi_Cal,Payer_Priv_Coverage,Payer_Workers_Comp,Payer_County_Indigent_Prog,Payer_Other_Government,Payer_Other_Indigent,Payer_Self_Pay,Payer_Other_Payer,Payer_Unknown,dx_Infectious,dx_Neoplasms,dx_Diseases_of_the_Blood,dx_Endocrine,dx_MentalHealth,dx_Nervous_System,dx_Eye,dx_Ear,dx_Circulatory,dx_Respiratory,dx_Digestive,dx_Skin,dx_Musculoskeletal,dx_Genitourinary,dx_Pregnancy_Childbirth,dx_Certain_Perinatal_Conditions,dx_Congenital,dx_Symptoms_Signs_NEC,dx_Injury_Poisoning,dx_Factors_Influencing_Health,dx_Other_Unknown,PR_Diagnostic_Therapeutic,PR_Obstetrical,PR_Cardiovascular_System,PR_Digestive_System,PR_Ear,PR_Endocrine_System,PR_Eye,PR_Reproductive_System,PR_Hemic_Lymph_System,PR_Integumentary_System,PR_Musculoskeletal_System,PR_Nervous_System,PR_Nose_Mouth_Pharynx,PR_Respiratory_System,PR_Urinary_System,PR_None,PR_No_Group_Assigned,Zip_Unk_Inv_Bl,Zip_Homeless,Zip_Foreign,Zip_Out_of_State,Zip_CA_Resident
0,106010735,10735,ALAMEDA,ALAMEDA HOSPITAL,2070 CLINTON AVE,ALAMEDA,94501,City or County,NaN,100-149,District 9,District 18,Alameda/Jack London Square/Oakland Inner Harbor,Urban,2646.0,130.0,NaN,NaN,1.0,1.0,1409,1368,1.0,NaN,NaN,104.0,187.0,296.0,408.0,590.0,570.0,619.0,4.0,487.0,2230.0,61.0,NaN,480.0,19.0,727.0,943.0,30.0,513.0,66.0,1823.0,5.0,461.0,380.0,12.0,94.0,NaN,NaN,NaN,3.0,NaN,NaN,2116.0,327.0,335.0,NaN,85839,2119.0,379.0,279.0,NaN,NaN,1.0,1166,204.0,60.0,574.0,NaN,NaN,119.0,102.0,500.0,NaN,NaN,46.0,6.0,NaN,1.0,NaN,NaN,109.0,2669,NaN,1478.0,973.0,204.0,1.0,48.0,19.0,NaN,55.0,NaN,NaN,460.0,49.0,52.0,138.0,79.0,181.0,4.0,6.0,470.0,272.0,343.0,73.0,135.0,151.0,NaN,NaN,NaN,128.0,217.0,17.0,3.0,321.0,NaN,221.0,168.0,NaN,NaN,1.0,19.0,NaN,39.0,123.0,28.0,3.0,78.0,11.0,1743.0,23.0,1.0,141.0,NaN,27.0,2609
1,106010739,10739,ALAMEDA,ALTA BATES SUMMIT MEDICAL CENTER-ALTA BATES CAMPUS,2450 ASHBY AVE,BERKELEY,94705,Non-profit Corporation (incl. Church-related),NaN,300-499,District 9,District 15,Albany/Berkeley East and North/Claremont/Cragmont/Forest Park/Rockridge/Temescal/Thousand Oaks,Urban,15007.0,NaN,NaN,NaN,NaN,NaN,5171,9836,NaN,4314.0,188.0,1720.0,3212.0,931.0,788.0,1104.0,1436.0,1314.0,NaN,4211.0,10314.0,482.0,44.0,1873.0,389.0,3099.0,5072.0,320.0,3941.0,269.0,9681.0,107.0,829.0,150.0,20.0,3.0,3.0,1.0,NaN,1.0,4212.0,NaN,4758.0,547.0,9702.0,NaN,60441,3534.0,4666.0,2595.0,4212.0,NaN,NaN,11358,274.0,118.0,922.0,40.0,2.0,260.0,189.0,1590.0,NaN,22.0,122.0,101.0,NaN,9.0,NaN,NaN,645.0,14362,NaN,3074.0,6095.0,5174.0,12.0,NaN,396.0,NaN,247.0,9.0,NaN,1322.0,241.0,139.0,269.0,128.0,222.0,11.0,15.0,773.0,277.0,801.0,161.0,562.0,317.0,4719.0,4309.0,6.0,136.0,584.0,15.0,NaN,1265.0,4209.0,401.0,698.0,3.0,5.0,4.0,227.0,17.0,189.0,663.0,247.0,33.0,167.0,122.0,6484.0,273.0,32.0,381.0,6.0,69.0,14519


## Join Data

Initialize a base dataframe of California acute care hospitals from the CMS data.

In [14]:
df = cms_hospitals[(cms_hospitals['state'] == 'CA') & (cms_hospitals['hospital_type'] == 'Acute Care Hospitals')].copy().add_prefix('cms_')
print(f'Number of records: {df.shape[0]:,}')

Number of records: 281


In [15]:
df.head(2)

,cms_record_number,cms_facility_id,cms_facility_name,cms_address,cms_citytown,cms_state,cms_zip_code,cms_countyparish,cms_telephone_number,cms_hospital_type,cms_hospital_ownership,cms_emergency_services,cms_meets_criteria_for_promoting_interoperability_of_ehrs,cms_meets_criteria_for_birthing_friendly_designation,cms_hospital_overall_rating,cms_hospital_overall_rating_footnote,cms_mort_group_measure_count,cms_count_of_facility_mort_measures,cms_count_of_mort_measures_better,cms_count_of_mort_measures_no_different,cms_count_of_mort_measures_worse,cms_mort_group_footnote,cms_safety_group_measure_count,cms_count_of_facility_safety_measures,cms_count_of_safety_measures_better,cms_count_of_safety_measures_no_different,cms_count_of_safety_measures_worse,cms_safety_group_footnote,cms_readm_group_measure_count,cms_count_of_facility_readm_measures,cms_count_of_readm_measures_better,cms_count_of_readm_measures_no_different,cms_count_of_readm_measures_worse,cms_readm_group_footnote,cms_pt_exp_group_measure_count,cms_count_of_facility_pt_exp_measures,cms_pt_exp_group_footnote,cms_te_group_measure_count,cms_count_of_facility_te_measures,cms_te_group_footnote
319,1596,050002,ST ROSE HOSPITAL,27200 CALAROGA AVE,HAYWARD,CA,94545,ALAMEDA,(510) 782-6200,Acute Care Hospitals,Voluntary non-profit - Church,No,Y,Y,1,,7,5,0,4,1,,8,5,0,5,0,,11,7,0,6,1,,8,8,,12,7,
320,1601,050006,PROVIDENCE ST JOSEPH HOSPITAL,2700 DOLBEER ST,EUREKA,CA,95501,HUMBOLDT,(707) 445-8121,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Y,Y,4,,7,7,0,7,0,,8,7,2,5,0,,11,11,0,11,0,,8,8,,12,11,


### Fuzzy Matching

The CMS and HCAI datasets use different hospital identifiers, requiring extra steps to join the data. Use fuzzy matching to match the datasets based on hospital name and address.

Combine the name and address fields to facilitate matching.

In [16]:
cms_name_address = df[['cms_facility_id', 'cms_facility_name', 'cms_address', 'cms_citytown', 'cms_zip_code']].copy()
cms_name_address['cms_name_address'] = cms_name_address['cms_facility_name'] + ', ' + cms_name_address['cms_address'] + ', ' + cms_name_address['cms_citytown'] + ', ' + cms_name_address['cms_zip_code']
cms_name_address['cms_name_address'] = cms_name_address['cms_name_address'].str.upper()
cms_name_address = cms_name_address[['cms_facility_id', 'cms_name_address']].copy()

hcai_name_address = hcai_facilities[['OSHPD_ID', 'FACILITY_NAME', 'DBA_ADDRESS1', 'DBA_CITY', 'DBA_ZIP_CODE']][hcai_facilities['LICENSE_TYPE_DESC'] == 'Hospital'].copy()
hcai_name_address['hcai_name_address'] = hcai_name_address['FACILITY_NAME'] + ', ' + hcai_name_address['DBA_ADDRESS1'] + ', ' + hcai_name_address['DBA_CITY'] + ', ' + hcai_name_address['DBA_ZIP_CODE'].astype(str).str.replace('\.0','', regex=True)
hcai_name_address['hcai_name_address'] = hcai_name_address['hcai_name_address'].str.upper()
hcai_name_address = hcai_name_address[['OSHPD_ID', 'hcai_name_address']]

Define a function that uses `process.extractOne` from the `fuzzywuzzy` library to find the best match for each record from one dataframe in another dataframe.

In [17]:
def fuzzy_merge(df1, df2, key1, key2, threshold=90):
    '''
    Function to merge two pandas dataframes based on a fuzzy match of text data.
    df1, df2: the two pandas dataframes that you want to merge using fuzzy matching
    key1, key2: the columns from the dataframes that the fuzzy matching will be based on
    threshold: similarity score threshold below which matches are considered unsuccessful
    limit: maximum number of matches to find for each row in df1
    '''
    matches = []
    for index, row in df1.iterrows():
        match = process.extractOne(row[key1], df2[key2], scorer=fuzz.token_sort_ratio, score_cutoff=threshold)
        if match:
            matches.append((index, match[2], match[1]))
    df_matches = pd.DataFrame(matches, columns=['index_df1', 'index_df2', 'match_score'])
    temp = df1.merge(df_matches, left_index=True, right_on='index_df1')
    temp = temp.merge(df2, left_on='index_df2', right_index=True).drop(['index_df1', 'index_df2'], axis=1)
    return temp

Run the function on our data to find the best match for each record from the CMS dataframe in the HCAI dataframe.

In [18]:
matched = fuzzy_merge(cms_name_address, hcai_name_address, 'cms_name_address', 'hcai_name_address', threshold=0)

Evaluate the distribution of the match scores.

In [19]:
matched['match_score'].describe()

count    281.000000
mean      96.088968
std        5.402637
min       62.000000
25%       95.000000
50%       98.000000
75%      100.000000
max      100.000000
Name: match_score, dtype: float64

Since there are a relatively small number of records in the dataset, and a small number of the records have low match scores, we will do some manual review to see if there are matches we should throw out.

In [20]:
matched[matched['match_score'] < 90].sort_values('match_score', ascending=False)

,cms_facility_id,cms_name_address,match_score,OSHPD_ID,hcai_name_address
113,050257,"GOOD SAMARITAN HOSPITAL, 901 OLIVE DRIVE, BAKERSFIELD, 93308",89,106150775,"GOOD SAMARITAN HOSPITAL-BAKERSFIELD, 901 OLIVE DR, BAKERSFIELD, 93308"
236,050704,"MISSION COMMUNITY HOSPITAL, 14850 ROSCOE BLVD, PANORAMA CITY, 91402",89,106190524,"MISSION COMMUNITY HOSPITAL - PANORAMA CAMPUS, 14850 ROSCOE BLVD, PANORAMA CITY, 91402"
122,050283,"STANFORD HEALTH CARE - VALLEYCARE, 5555 WEST LAS POSITAS BOULEVARD, PLEASANTON, 94588",89,106014050,"STANFORD HEALTH CARE TRI-VALLEY, 5555 W LAS POSITAS BLVD, PLEASANTON, 94588"
78,050149,"CALIFORNIA HOSPITAL MEDICAL CENTER LA, 1401 SOUTH GRAND AVENUE, LOS ANGELES, 90015",89,106190125,"CALIFORNIA HOSPITAL MEDICAL CENTER - LOS ANGELES, 1401 S GRAND AVE, LOS ANGELES, 90015"
120,050280,"MERCY MEDICAL CENTER REDDING, 2175 ROSALINE AVE, CLAIRMONT HGTS, REDDING, 96001",89,106450949,"MERCY MEDICAL CENTER - REDDING, 2175 ROSALINE AVE, REDDING, 96001"
103,050234,"SHARP CORONADO HOSPITAL AND HLTHCR CTR, 250 PROSPECT PLACE, CORONADO, 92118",88,106370689,"SHARP CORONADO HOSPITAL AND HEALTHCARE CENTER, 250 PROSPECT PL, CORONADO, 92118"
129,050300,"ST MARY MEDICAL CENTER, 18300 HIGHWAY 18, APPLE VALLEY, 92307",88,106361343,"ST. MARY MEDICAL CENTER - APPLE VALLEY, 18300 US HIGHWAY 18, APPLE VALLEY, 92307"
127,050295,"MERCY HOSPITAL, 2215 TRUXTUN AVENUE, BAKERSFIELD, 93301",88,106150761,"MERCY HOSPITAL - BAKERSFIELD, 2215 TRUXTUN AVE, BAKERSFIELD, 93301"
13,050030,"OROVILLE HOSPITAL, 2767 OLIVE HIGHWAY, OROVILLE, 95966",88,106040937,"OROVILLE HOSPITAL, 2767 OLIVE HWY, OROVILLE, 95966"
93,050197,"SEQUOIA HOSPITAL, 170 ALAMEDA DE LAS PULGAS, REDWOOD CITY, 94062",87,106410891,"SEQUOIA HOSPITAL, 170 ALAMEDA, REDWOOD CITY, 94062"


The matches seem to deteriorate for the small number of records below a match score of 75. Merge only records with a score of 75 or greater with the base dataframe. Keep only records with a match.

In [21]:
df = df.merge(matched[matched['match_score'] >= 75], on='cms_facility_id', how='inner')
print(f'Number of records: {df.shape[0]:,}')

Number of records: 276


### Regular Joins

Join selected additional CMS and HCAI data on the relevant ID.

In [22]:
df = df.merge(cms_hvbp[['facility_id', 'fiscal_year', 'weighted_normalized_clinical_outcomes_domain_score', 'unweighted_normalized_clinical_outcomes_domain_score']], left_on='cms_facility_id', right_on='facility_id', how='left')
df = df.merge(hcai_facilities[['OSHPD_ID', 'FACILITY_LEVEL_DESC', 'ER_SERVICE_LEVEL_DESC', 'TOTAL_NUMBER_BEDS', 'LICENSE_CATEGORY_DESC']], on='OSHPD_ID', how='left')
df = df.merge(hcai_inpatients[['oshpd_id9', 'CONTROL_TYPE_DESC', 'teach_rural', 'licensed_bed_size', 'mssa_designation',
                               'Sex_Male', 'Sex_Female', 'Sex_Other_Unknown',
                               'Age_0_09', 'Age_10_19', 'Age_20_29', 'Age_30_39', 'Age_40_49', 'Age_50_59', 'Age_60_69', 'Age_70_79', 'Age_80_', 'Age_Other_Unknown']], left_on='OSHPD_ID', right_on='oshpd_id9', how='left')
print(f'Number of records: {df.shape[0]:,}')

Number of records: 276


In [23]:
df.head(2)

,cms_record_number,cms_facility_id,cms_facility_name,cms_address,cms_citytown,cms_state,cms_zip_code,cms_countyparish,cms_telephone_number,cms_hospital_type,cms_hospital_ownership,cms_emergency_services,cms_meets_criteria_for_promoting_interoperability_of_ehrs,cms_meets_criteria_for_birthing_friendly_designation,cms_hospital_overall_rating,cms_hospital_overall_rating_footnote,cms_mort_group_measure_count,cms_count_of_facility_mort_measures,cms_count_of_mort_measures_better,cms_count_of_mort_measures_no_different,cms_count_of_mort_measures_worse,cms_mort_group_footnote,cms_safety_group_measure_count,cms_count_of_facility_safety_measures,cms_count_of_safety_measures_better,cms_count_of_safety_measures_no_different,cms_count_of_safety_measures_worse,cms_safety_group_footnote,cms_readm_group_measure_count,cms_count_of_facility_readm_measures,cms_count_of_readm_measures_better,cms_count_of_readm_measures_no_different,cms_count_of_readm_measures_worse,cms_readm_group_footnote,cms_pt_exp_group_measure_count,cms_count_of_facility_pt_exp_measures,cms_pt_exp_group_footnote,cms_te_group_measure_count,cms_count_of_facility_te_measures,cms_te_group_footnote,cms_name_address,match_score,OSHPD_ID,hcai_name_address,facility_id,fiscal_year,weighted_normalized_clinical_outcomes_domain_score,unweighted_normalized_clinical_outcomes_domain_score,FACILITY_LEVEL_DESC,ER_SERVICE_LEVEL_DESC,TOTAL_NUMBER_BEDS,LICENSE_CATEGORY_DESC,oshpd_id9,CONTROL_TYPE_DESC,teach_rural,licensed_bed_size,mssa_designation,Sex_Male,Sex_Female,Sex_Other_Unknown,Age_0_09,Age_10_19,Age_20_29,Age_30_39,Age_40_49,Age_50_59,Age_60_69,Age_70_79,Age_80_,Age_Other_Unknown
0,1596,050002,ST ROSE HOSPITAL,27200 CALAROGA AVE,HAYWARD,CA,94545,ALAMEDA,(510) 782-6200,Acute Care Hospitals,Voluntary non-profit - Church,No,Y,Y,1,,7,5,0,4,1,,8,5,0,5,0,,11,7,0,6,1,,8,8,,12,7,,"ST ROSE HOSPITAL, 27200 CALAROGA AVE, HAYWARD, 94545",100,106010967,"ST. ROSE HOSPITAL, 27200 CALAROGA AVE, HAYWARD, 94545",NaN,NaN,NaN,NaN,Parent Facility,Emergency - Basic,171,General Acute Care Hospital,106010967.0,Non-profit Corporation (incl. Church-related),Teaching,150-199,Urban,2084.0,2125.0,NaN,315.0,52.0,331.0,368.0,361.0,534.0,878.0,690.0,680.0,NaN
1,1601,050006,PROVIDENCE ST JOSEPH HOSPITAL,2700 DOLBEER ST,EUREKA,CA,95501,HUMBOLDT,(707) 445-8121,Acute Care Hospitals,Voluntary non-profit - Private,Yes,Y,Y,4,,7,7,0,7,0,,8,7,2,5,0,,11,11,0,11,0,,8,8,,12,11,,"PROVIDENCE ST JOSEPH HOSPITAL, 2700 DOLBEER ST, EUREKA, 95501",94,106121080,"PROVIDENCE ST. JOSEPH HOSPITAL - EUREKA, 2700 DOLBEER ST, EUREKA, 95501",050006,2023,13.500000000000,54.000000000000,Parent Facility,Emergency - Basic,138,General Acute Care Hospital,106121080.0,Investor - Limited Liability Company,Small/Rural,100-149,Rural,3430.0,3778.0,NaN,784.0,68.0,573.0,732.0,532.0,760.0,1358.0,1378.0,1023.0,NaN


## Clean Data

### Construct the Label

The label is a binary indicator of whether the hospital participated in the CMS FY23 Hospital Value-Based Purchasing (HVBP) Program. The Hospital Value-Based Purchasing (HVBP) - Total Performance Score dataset provides a list of all hospitals participating, per the documentation at https://data.cms.gov/provider-data/dataset/ypbt-wvdk. Therefore we will define participation as having a record in that dataset.

In [24]:
df.loc[df['fiscal_year'] == "2023", 'label'] = 1
df.loc[df['fiscal_year'].isnull(), 'label'] = 0
pd.crosstab(df['fiscal_year'].fillna("N/A"), df['label'], dropna=False)

label,0.0,1.0
fiscal_year,,
2023,0,237
N/A,39,0


### Construct the Features

Recode hospital ownership.

In [25]:
df['cms_hospital_ownership'] = np.where(df['cms_hospital_ownership'].str.contains('Voluntary non-profit'), 'Voluntary Non-Profit', df['cms_hospital_ownership'])
df['cms_hospital_ownership'] = np.where(df['cms_hospital_ownership'].str.contains('Government'), 'Government', df['cms_hospital_ownership'])
df['cms_hospital_ownership'] = np.where(df['cms_hospital_ownership'] == 'Veterans Health Administration', 'Government', df['cms_hospital_ownership'])
df['cms_hospital_ownership'] = np.where(df['cms_hospital_ownership'] == 'Physician', 'Proprietary', df['cms_hospital_ownership'])

Recode whether has emergency services.

In [26]:
df['cms_emergency_services'] = np.where(df['cms_emergency_services'] == 'Yes', 1, 0)

Recode whether meets criteria for promoting interoperability of EHRs.

In [27]:
df['cms_meets_criteria_for_promoting_interoperability_of_ehrs'] = np.where(df['cms_meets_criteria_for_promoting_interoperability_of_ehrs'] == 'Y', 1, 0)

Recode whether is a teaching hospital.

In [28]:
df['teaching'] = np.where(df['teach_rural'] == 'Teaching', 1, 0)

Recode whether is in a rural area. There are a small number of missing values and we will impute these by assuming they are non-rural.

In [29]:
df['mssa_designation'].value_counts(dropna=False)

mssa_designation
Urban    244
Rural     30
NaN        2
Name: count, dtype: int64

In [30]:
df['rural'] = np.where((df['mssa_designation'] == 'Rural') | (df['mssa_designation'] == 'Frontier'), 1, 0)

Convert number of beds to numeric.

In [31]:
df['TOTAL_NUMBER_BEDS'] = df['TOTAL_NUMBER_BEDS'].str.replace(',','').astype(int)

### Additional Preprocessing

Keep only fields of interest.

In [32]:
df = df[['label', 'cms_hospital_ownership', 'cms_emergency_services', 'cms_meets_criteria_for_promoting_interoperability_of_ehrs',
         'teaching', 'rural', 'TOTAL_NUMBER_BEDS']]
df = df.rename(columns={'cms_hospital_ownership':'ownership','cms_emergency_services':'emergency',
                        'cms_meets_criteria_for_promoting_interoperability_of_ehrs':'interoperability','TOTAL_NUMBER_BEDS':'beds'})
df.head(2)

,label,ownership,emergency,interoperability,teaching,rural,beds
0,0.0,Voluntary Non-Profit,0,1,1,0,171
1,1.0,Voluntary Non-Profit,1,1,0,1,138


Upsample the minority class to address class imbalance.

In [33]:
class_0 = df[df['label'] == 0]
class_1 = df[df['label'] == 1]

upsampled_class_0 = class_0.sample(n=class_1.shape[0], replace=True, random_state=42)
df = pd.concat([class_1, upsampled_class_0]).sample(frac=1, random_state=42).reset_index(drop=True)
df['label'].value_counts(dropna=False)

label
0.0    237
1.0    237
Name: count, dtype: int64

Split the data into training (70%), validation (15%), and test (15%) sets.

In [34]:
X = df.drop('label', axis=1)
y = df['label']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

One-hot encode categorical features, and scale the numeric feature. We will do this using a pipeline to make the transformations replicable. Note that we use `fit_transform` with the training set but just `transform` with the validation and test sets to ensure that the transformations learned on the training set are applied consistently.

In [35]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['ownership', 'emergency', 'interoperability', 'teaching', 'rural']),
        ('num', MinMaxScaler(), ['beds'])
    ])

In [36]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
])

In [37]:
X_train = pipeline.fit_transform(X_train)
X_val = pipeline.transform(X_val)
X_test = pipeline.transform(X_test)

Save the fitted pipeline for later use.

In [38]:
with open('predicting_medicare_vbp_pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

## Develop a Deep Learning Model

Use Keras (a high-level API that runs on TensorFlow) to train and test a deep learning model to predict whether a hospital participated in the HBPV program.

Build and compile the model.
- `Sequential()` creates a linear stack of layers for building a neural network.
- `Dense()` creates fully connected dense layers where each neuron is connected to each neuron in the preceding layer. This model consists of one input layer, one hidden layer, and an output layer. The output layer uses the `sigmoid` activation function to put the output into the range 0 to 1 for binary classification.
- `compile()` configures the model for training including the optimizer, loss function, and metric for evaluation.

In [39]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Define a `ModelCheckpoint` callback to save the best model during training. We will save only the best model based on minimizing validation loss, and print the progress information to the console. The `.h5` format is commonly used to save Keras models.

In [40]:
checkpoint_callback = ModelCheckpoint('predicting_medicare_vbp_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

Train and save the model.
- `epochs` refers to the number of times that the model will go through the training dataset.
- `batch_size` refers to the number of samples that will be processed before the model weights are updated.
- `callbacks` uses the `ModelCheckpoint` that we defined to save the best model.

In [41]:
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_val, y_val), callbacks=[checkpoint_callback])

Epoch 1/20
 1/21 [>.............................] - ETA: 11s - loss: 0.6972 - accuracy: 0.5000
Epoch 1: val_loss improved from inf to 0.65540, saving model to predicting_medicare_vbp_model.h5
21/21 [==============================] - 1s 10ms/step - loss: 0.6846 - accuracy: 0.5861 - val_loss: 0.6554 - val_accuracy: 0.6620
Epoch 2/20
 1/21 [>.............................] - ETA: 0s - loss: 0.6779 - accuracy: 0.5625
Epoch 2: val_loss improved from 0.65540 to 0.60144, saving model to predicting_medicare_vbp_model.h5
21/21 [==============================] - 0s 3ms/step - loss: 0.6305 - accuracy: 0.7039 - val_loss: 0.6014 - val_accuracy: 0.7606
Epoch 3/20
 1/21 [>.............................] - ETA: 0s - loss: 0.5846 - accuracy: 0.8125
Epoch 3: val_loss improved from 0.60144 to 0.56059, saving model to predicting_medicare_vbp_model.h5
21/21 [==============================] - 0s 3ms/step - loss: 0.5825 - accuracy: 0.7583 - val_loss: 0.5606 - val_accuracy: 0.7606
Epoch 4/20
 1/21 [>...........

Evaluate the model on the test set. We will use a threshold of 0.5 (a typical default) to separate predicted probabilities into classes of 0 and 1. In the confusion matrix, upper left is true negative (TN), bottom left is false negative (FN), upper right is false positive (FP), and bottom right is true positive (TP).

In [42]:
predictions = model.predict(X_test)
y_pred_binary = (predictions > 0.5).astype(int)

confusion_matrix(y_test, y_pred_binary)

3/3 [==============================] - 0s 2ms/step


array([[19, 17],
       [ 7, 29]])

In the classification report, we are primarily interested in the values for the positive class (top section, row 1.0).
- `precision` is the ratio of correctly predicted positive predictions (TP) to the total predicted positives (TP + FP)
- `recall` is the ratio of correctly predicted positive predictions (TP) to the total actual positives (TP + FN). Recall is also known as sensitivity or the true positive rate.
- `f1-score` is the harmonic mean of precision and recall.
- `support` is the number of actual occurrences of each class (0 and 1) in the dataset.
- `accuracy` is the ratio of correctly predicted observations (TP + TN) to the total number of observations.
- `macro avg` is the unweighted average of the relevant metric across all classes.
- `weighted avg` is the average of the relevant metric weighted by the number of samples in each class.

In [43]:
print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

         0.0       0.73      0.53      0.61        36
         1.0       0.63      0.81      0.71        36

    accuracy                           0.67        72
   macro avg       0.68      0.67      0.66        72
weighted avg       0.68      0.67      0.66        72



## Develop a Streamlit App

To run the app locally, copy the code below into a `.py` file. In a terminal, navigate to where the `.py` file is saved. Enter the command `streamlit run [filename].py` to launch the app. Ensure the relevant packages are available in the environment being used in the terminal.

Useful resources:
- https://blog.streamlit.io/host-your-streamlit-app-for-free/#:~:text=Connect%20your%20account%20to%20GitHub,-There%20are%20two&text=This%20will%20let%20Community%20Cloud,%2C%20click%20%E2%80%9CAuthorize%20streamlit.%22&text=Now%20you're%20ready%20to,let's%20create%20a%20GitHub%20repo
- https://dev.to/surendraredd/how-to-deploy-streamlitapp-2p53

In [46]:
import pickle
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf
import streamlit as st
import pandas as pd

with open('predicting_medicare_vbp_pipeline.pkl', 'rb') as file:
    pipeline = pickle.load(file)

model = tf.keras.models.load_model('predicting_medicare_vbp_model.h5')

st.title('Predicting Medicare Value-Based Payment Program Participation')

st.write("This app predicts whether a hospital is likely to participate in the Medicare Hospital Value-Based Purchasing (HVBP) Program with the Centers for Medicare & Medicaid Services (CMS). The training set consisted of acute care hospitals in California.")

category_options = {
    'ownership': ["Voluntary Non-Profit", "Proprietary", "Government"],
    'emergency': ["Yes", "No"],
    'interoperability': ["Yes", "No"],
    'teaching': ["Yes", "No"],
    'rural': ["Yes", "No"]
}

col1, col2 = st.columns(2)

with col1:
    ownership = st.selectbox('Hospital Ownership', list(category_options['ownership']))
    emergency = st.selectbox('Has Emergency Services', list(category_options['emergency']))
    interoperability = st.selectbox('Meets Criteria for Promoting Interoperability of EHRs', list(category_options['interoperability']))

with col2:
    teaching = st.selectbox('Teaching Hospital', list(category_options['teaching']))
    rural = st.selectbox('Rural Hospital', list(category_options['rural']))
    beds = st.slider('Number of Beds', min_value=0, max_value=1400, value=200)

input = pd.DataFrame({
    'ownership': [ownership],
    'emergency': [1 if emergency == 'Yes' else 0],
    'interoperability': [1 if interoperability == 'Yes' else 0],
    'teaching': [1 if teaching == 'Yes' else 0],
    'rural': [1 if rural == 'Yes' else 0],
    'beds': [beds]
})

input = pipeline.transform(input)

probability = model.predict(input)
if probability[0][0] > 0.8:
    predicted_class = "Very Likely to Participate"
elif probability[0][0] > 0.5:
    predicted_class = "Somewhat Likely to Participate"
else:
    predicted_class = "Not Likely to Participate"
st.success(f'The Hospital is {predicted_class} (Probability of Participation = {probability[0][0]:.3f})')

1/1 [==============================] - 0s 207ms/step


DeltaGenerator()